In [138]:
import csv
import json
import collections

import pandas as pd
import numpy as np

In [139]:
df = pd.read_csv('../Downloads/Batch_3783234_batch_results.csv')

In [140]:
def get_number_urls_with_undecided(df):
    data = {}
    for index, row in df.iterrows():
        url = row[df.columns.get_loc('Input.url')]
        cat = row[df.columns.get_loc('Answer.category.label')]
        if url not in data:
            data[url] = []
        data[url].append(cat)
    
    num_with_undecided = 0
    undecided = set()
    decided = set()
    for url, cats in data.items():
        counter = collections.Counter(cats)
        sorted_by = counter.most_common()
        
        max_votes = sorted_by[0][1]
        if(len(sorted_by) > 1 and sorted_by[1][1] == max_votes):
            num_with_undecided += 1
            undecided.add(url)
        else:
            decided.add(url)
        
    return decided, undecided, num_with_undecided, (2*float(num_with_undecided)/df.shape[0])*100.0

In [141]:
def average_time_taken_for_undecided(df, undecided, workers):
    
    time_taken = 0.0
    num_of_undecided = len(undecided)
    for url in undecided:
        rows = df.loc[df['Input.url'] == url]
        for index, row in rows.iterrows():
            time_taken += row[df.columns.get_loc('WorkTimeInSeconds')]
    return time_taken/(workers*num_of_undecided)

In [142]:
decided, undecided, num_with_undecided, percentage_undecided = get_number_urls_with_undecided(df)
average_time_undecided = average_time_taken_for_undecided(df, undecided, 2)
average_time_decided = average_time_taken_for_undecided(df, decided, 2)

In [143]:
print("Batch 1: 44 Unique Urls, 5 lesson plans")
print("Number of Undecided Urls: "+str(num_with_undecided))
print("Percentage of Undecided Urls: "+str(percentage_undecided))
print("Average Time Taken for Undecided Urls: "+str(average_time_undecided))
print("Average Time Taken for Decided Urls: "+str(average_time_decided))

Batch 1: 44 Unique Urls, 5 lesson plans
Number of Undecided Urls: 27
Percentage of Undecided Urls: 61.3636363636
Average Time Taken for Undecided Urls: 44.7777777778
Average Time Taken for Decided Urls: 44.0625


In [145]:
df.shape

(88, 33)